<a href="https://colab.research.google.com/github/NobuoTsukamoto/tensorrt-examples/blob/main/python/detection/Add_TFLiteNMS_Plugin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert a SSDLite MobileNet V2 TFLite model to ONNX and Add TFLite NMS Plugin.

This notebook converts from TensorFlow Lite model to ONNX model and replaces NonMax Suppression with TensorRT's TF-Lite NMS Plugin.  
The model uses [SSD Lite MobileNet V2](https://github.com/tensorflow/models/blob/a4fd64722dcdd42361beb1be478ad8fdb10bde31/research/object_detection/g3doc/tf1_detection_zoo.md) and modifies the ONNX model with [ONNX GraphSurgeon](https://github.com/NVIDIA/TensorRT/tree/master/tools/onnx-graphsurgeon).

Copyright (c) 2021 Nobuo Tsukamoto  
  
This software is released under the MIT License.  
See the LICENSE file in the project root for more information.


## Reference
- [onnx/tensorflow-onnx - Convert a mobiledet tflite model to ONNX](https://github.com/onnx/tensorflow-onnx/blob/de67f2051d1f036b29901e2910c8eb41a1c71b6e/tutorials/mobiledet-tflite.ipynb)
- [TensorFlow Model Garden - TensorFlow 1 Detection Model Zoo](https://github.com/tensorflow/models/blob/a4fd64722dcdd42361beb1be478ad8fdb10bde31/research/object_detection/g3doc/tf1_detection_zoo.md)
- [TensorFlow Model Garden - Running on mobile with TensorFlow Lite](https://github.com/tensorflow/models/blob/a4fd64722dcdd42361beb1be478ad8fdb10bde31/research/object_detection/g3doc/running_on_mobile_tensorflowlite.md)
- [TensorRT Backend For ONNX](https://github.com/onnx/onnx-tensorrt/tree/868e636f51f0d7e61df340371303275265146fe0)
- [ONNX GraphSurgeon](https://github.com/NVIDIA/TensorRT/tree/0953f2ff8762b28e0f1bef0582b6ca3d7a12fcaa/tools/onnx-graphsurgeon)

## Convert the TensorFlow Lite Model.

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


Clone [tensorflow/models](https://github.com/tensorflow/models) repository and install dependency.

In [2]:
%%bash

pip install tensorflow-addons
git clone https://github.com/tensorflow/models.git
cd models
git checkout a4fd64722dcdd42361beb1be478ad8fdb10bde31

cd research
protoc object_detection/protos/*.proto --python_out=.

cp object_detection/packages/tf1/setup.py .
python -m pip install  .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1657627 sha256=7b19e961b8a0662585209cca8398fe4e5a8519ba2561ae264fb607497b10fa2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-omepu6j0/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


Cloning into 'models'...
Note: checking out 'a4fd64722dcdd42361beb1be478ad8fdb10bde31'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a4fd6472 Internal change


In [3]:
import os

os.environ['PYTHONPATH'] = '/content/models:' + os.environ['PYTHONPATH']
print(os.environ['PYTHONPATH'])

/content/models:/tensorflow-1.15.2/python3.7:/env/python


In [4]:
%cd models/research/
!python object_detection/builders/model_builder_tf1_test.py

/content/models/research
Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_faster

Download SSDLite MobileNet V2 checkpoint and export TensorFlow Lite model.

In [8]:
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz -P /content
!tar xf /content/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz -C /content

--2021-06-30 11:09:23--  http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.126.128, 2a00:1450:4013:c01::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.126.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51025348 (49M) [application/x-tar]
Saving to: ‘/content/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz.1’

ssdlite_mobilenet_v 100%[===================>]  48.66M   155MB/s    in 0.3s    

2021-06-30 11:09:23 (155 MB/s) - ‘/content/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz.1’ saved [51025348/51025348]



In [9]:
!python object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path="/content/ssdlite_mobilenet_v2_coco_2018_05_09/pipeline.config" \
    --trained_checkpoint_prefix="/content/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt" \
    --output_directory="/content/ssdlite_mobilenet_v2_coco_2018_05_09/tflite" \
    --add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0630 11:09:33.057559 139931898357632 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0630 11:09:35.229738 139931898357632 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0630 11:09:35.306802 139931898357632 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0630 11:09:35.391663 139931898357632 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [10]:
!tflite_convert \
    --enable_v1_converter \
    --graph_def_file="/content/ssdlite_mobilenet_v2_coco_2018_05_09/tflite/tflite_graph.pb" \
    --output_file="/content/ssdlite_mobilenet_v2_coco_2018_05_09/tflite/ssdlite_mobilenet_v2_320x320.tflite" \
    --inference_input_type=FLOAT \
    --inference_type=FLOAT \
    --input_arrays="normalized_input_image_tensor" \
    --output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
    --input_shapes=1,300,300,3 \
    --allow_nudging_weights_to_use_fast_gemm_kernel=true \
    --allow_custom_op

2021-06-30 11:09:53.163088: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-06-30 11:09:53.184290: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-06-30 11:09:53.184868: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla V100-SXM2-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:00:04.0
2021-06-30 11:09:53.185132: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-06-30 11:09:53.186718: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-06-30 11:09:53.194978: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft

## Export ONNX Model

Install dependency.

In [11]:
%%bash
pip3 install onnxruntime
pip3 install tf2onnx

Note: TensorRT 7.2 supports operators up to Opset 13.  
- [onnx/onnx-tensorrt - Supported ONNX Operators](https://github.com/onnx/onnx-tensorrt/blob/868e636f51f0d7e61df340371303275265146fe0/docs/operators.md)

In [12]:
!python3 -m tf2onnx.convert --opset 11 \
    --tflite /content/ssdlite_mobilenet_v2_coco_2018_05_09/tflite/ssdlite_mobilenet_v2_320x320.tflite \
    --output /content/ssdlite_mobilenet_v2_coco_2018_05_09/onnx/ssdlite_mobilenet_v2_320x320.onnx

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))

2021-06-30 11:18:46,649 - WARNING - From /usr/local/lib/python3.7/dist-packages/tf2onnx/verbose_logging.py:76: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

2021-06-30 11:18:46,650 - INFO - Using tensorflow=1.15.2, onnx=1.9.0, tf2onnx=1.8.5/50049d
2021-06-30 11:18:46,650 - INFO - Using opset <onnx, 13>
2021-06-30 11:18:47,078 - WARNING - NMS node TFLite_Detection_PostProcess uses fast NMS. ONNX will approximate with standard NMS.
2021-06-30 11:18:47,112 - INFO - Optimizing ONNX model
2021-06-30 11:18:48,579 - INFO - After optimization: Cast -13 (15->2), Const -180 (377->197), Identity -4 (4->0), Reshape -33 (46->13), Transpose -341 (353->12)
2021-06-30 11:18:48,622 - INFO - 
2021-06-30 11:18:

## Add TF-Lite NMS Plugin

In [13]:
!python3 -m pip install onnx_graphsurgeon --index-url https://pypi.ngc.nvidia.com

Looking in indexes: https://pypi.ngc.nvidia.com


In [14]:
%cd /content/
!git clone https://github.com/NobuoTsukamoto/tensorrt-examples

/content
Cloning into 'tensorrt-examples'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17 (delta 1), reused 13 (delta 1), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [15]:
%cd /content/tensorrt-examples/python/detection

/content/tensorrt-examples/python/detection


In [16]:
!python3 add_tensorrt_tflitenms_plugin.py \
    --input /content/ssdlite_mobilenet_v2_coco_2018_05_09/onnx/ssdlite_mobilenet_v2_320x320.onnx \
    --output /content/ssdlite_mobilenet_v2_coco_2018_05_09/onnx/ssdlite_mobilenet_v2_320x320_gs.onnx

{'max_classes_per_detection': 1, 'max_detections': 10, 'back_ground_Label_id': 0, 'nms_iou_threshold': 0.6, 'nms_score_threshold': 0.1, 'num_classes': 91, 'y_scale': 10.0, 'x_scale': 10.0, 'h_scale': 5.0, 'w_scale': 5.0, 'scoreBits': 16}
Saving the ONNX model to /content/ssdlite_mobilenet_v2_coco_2018_05_09/onnx/ssdlite_mobilenet_v2_320x320_gs.onnx
